# Data Analysis Demos

## Import Data

In [2]:
import pandas as pd
import sys
from decouple import config
from report_builder import *

print("Hello World!")
print(sys.executable)

DATA_FILE_PATH = config('DATA_FILE_PATH')
report = ReportBuilder(DATA_FILE_PATH, 2024)

df = report.main_df
df.head()

Hello World!
/usr/local/bin/python


,Transaction Date,Amount,Transaction Type,Account,Description,Label
0,2024-01-01,-132.97,CREDIT,NEEDS,H-E-B #591 ROUND ROCK TX,NaN
1,2024-01-01,-5.86,CREDIT,NEEDS,H-E-B #034 ROUND ROCK TX,NaN
2,2024-01-01,-108.04,CREDIT,NEEDS,JONAH WATER SPECIAL UTILI512-759-1286 TX,NaN
3,2024-01-01,-6.01,CREDIT,WANTS,STARBUCKS 800-782-7282,NaN
5,2024-01-02,-2426.86,DEBIT,NEEDS,DOVENMUEHLE MTG MORTG PYMT 010124 1465440376 M...,Mortgage Payment


## Data Manipulation Demos

In [ ]:
print('------- Data Frame Info -------')
print(df.index)
print(df.columns)
print(df.ndim)
print(df.shape)
print(df.size)
print(df.size)
print('------- Memory Usage ------')
print(df.memory_usage())

In [ ]:
grp1 = df.groupby(Col.AccountType.value)
print(grp1.size())
# grp1.first()
# grp1.apply(list)
# grp1.groups


subgrp1 = grp1.get_group("NEEDS")
subgrp1.head()


In [ ]:
grp2 = df.groupby([Col.AccountType.value, Col.TransactionType.value])
print(grp2.size())
grp2.first()

# subgrp2 = grp2.get_group("NEEDS")
# subgrp2.head()



In [ ]:
grp3 = df.groupby([
            Col.AccountType.value,
            Col.TransactionType.value,
            df[Col.TransactionDate.value].dt.month,
        ])

# print(grp3.size())
grp3.first()
# grp3.groups

## Data Visualization Demo

In [ ]:
import plotly.express as px

# hello world figure
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.show()

In [ ]:
grp4 = df.groupby([
            Col.AccountType.value,
            Col.TransactionType.value,
        ])

grp4_grps = list(grp4.groups)
print(grp4_grps)
print(grp4_grps[0])
print(type(grp4_grps[0]))

# grp4.get_group(grp4_grps[0])

grp_tuple = ('NEEDS', 'DEBIT')
subgrp40 = grp4.get_group(grp_tuple) # issue with group by returning any instead of df type, may need to just filter before grouping


In [ ]:
import calendar
from datetime import datetime
from numpy import dtype

from pandas import DataFrame

def get_month_name(month_num: int):
    return calendar.month_name[month_num]

def get_df_grp(dfi: DataFrame, filterByFunc, grpByFunc):
    return dfi.loc[filterByFunc].groupby(grpByFunc)[Col.Amount.value]


# print(df[Col.CustomTags.value].head())
# print(df[Col.CustomTags.value].explode().head())
# print(df[Col.CustomTags.value].explode() == 'NOISE')
# print(df[Col.CustomTags.value].explode() != 'NOISE')

# print(df[Col.TransactionDate.value] >= datetime(2023, 1, 1))

qry5 = (df[Col.CustomTags.value].explode() != 'NOISE')\
        & (df[Col.TransactionDate.value] >= datetime(2023, 1, 1))\
        & (df[Col.TransactionDate.value] < datetime(2023, 10, 1))\
        & (df[Col.TransactionType.value] == 'DEBIT')
        # & (df[Col.AccountType.value] == 'NEEDS')\
        

filterGrp5 = df.loc[qry5]

# print(filterGrp5[Col.CustomTags.value].explode().unique()) 
# filterGrp5


xVals = filterGrp5[Col.TransactionDate.value].dt.month.unique() 
# print(xVals)
debit_per_month_group = pd.DataFrame(list(map(get_month_name, xVals)))
debit_per_month_group[1] = filterGrp5.loc[(df[Col.Amount.value] > 0)]\
                            .groupby(filterGrp5[Col.TransactionDate.value].dt.month)[Col.Amount.value]\
                            .sum()\
                            .values

debit_per_month_group[2] = filterGrp5.loc[(df[Col.Amount.value] < 0)]\
                            .groupby(filterGrp5[Col.TransactionDate.value].dt.month)[Col.Amount.value]\
                            .apply(lambda val: val.abs().sum())\
                            .values


debit_per_month_group.columns = ["Month", "Income", "Spent"]
debit_per_month_group

# Ideal Plot
# Month Name, Amount > 0, Amount < 0, Surplus / Deficit
fig = px.bar(debit_per_month_group, x="Month", y=debit_per_month_group.columns[1:], barmode='group')
fig.show()

# use Subplots of pie charts https://plotly.com/python/subplots/
# in order to show top spending per area per month (i.e. Top 3 Credited items as % of month)
# use plotly filter to filter those months

# use cards in streamlit to show KPIs (i.e. YTD Sum, Montly Avg, Quarterly Trend, Savings Rate)

## Fuzzy Match Demo

In [35]:
# Demo Matching Similar Charges from previous month, so that labels can be carried over automatically
from rapidfuzz import fuzz

# fuzz.ratio("this is a test", "this is a test!")

curr_month_index = 3
df_curr_month = df.loc[(df[Col.TransactionDate.value] >= datetime(2024, curr_month_index, 1)) & (df[Col.TransactionDate.value] < datetime(2024, curr_month_index + 1, 1))]
df_prev_month = df.loc[(df[Col.TransactionDate.value] >= datetime(2024, curr_month_index - 1, 1)) & (df[Col.TransactionDate.value] < datetime(2024, curr_month_index, 1)) & (df[Col.Label.value])]

# df_curr_month.head()
first_val = df_curr_month.iloc[1][Col.Description.value]

# df_prev_month
df_prev_month = df_prev_month.drop_duplicates(subset=[Col.Label.value])
# df_prev_month
df_prev_month_labels = df_prev_month[[Col.Label.value, Col.Description.value]].to_dict('records')

print(first_val)

fuzz_ratio_obj = { Col.Label.value: '',
                  'MaxFuzzRatio': 0 }
for row in df_prev_month_labels:
    fuzz_ratio = fuzz.ratio(first_val, row[Col.Description.value])
    row['FuzzRatio'] = fuzz_ratio
    print(row)
    if(fuzz_ratio > fuzz_ratio_obj['MaxFuzzRatio']):
        fuzz_ratio_obj = { Col.Label.value: row[Col.Label.value],
                           'MaxFuzzRatio': fuzz_ratio }

print(fuzz_ratio_obj)

# df_prev_month_labels


# df_fuzz_ratio = df_curr_month[Col.Description.value].apply(lambda val: '')
# df_fuzz_ratio
# df_curr_month.insert(6, 'FuzzRatio', ] = 
# df_curr_month







MONARCH BEHAVIOR PAYROLL 0YG37 A4CH0PNC1 AMAYA, STEPHANIE
{'Label': 'Mortgage Payment', 'Description': 'DOVENMUEHLE MTG MORTG PYMT 020124 1465440376 Michael Amaya', 'FuzzRatio': 31.304347826086953}
{'Label': "Stephanie's Income", 'Description': 'MONARCH BEHAVIOR PAYROLL 0YG37 A4CH0PNC1 AMAYA, STEPHANIE', 'FuzzRatio': 100.0}
{'Label': "Michael's Income", 'Description': 'Bullhorn Inc-OSV 0000461050 240215 10OSV0064l Michael Amaya', 'FuzzRatio': 22.413793103448278}
{'Label': 'Credit Card Payment', 'Description': 'CHASE CREDIT CRD EPAY 240227 7337682519 MICHAEL AMAYA', 'FuzzRatio': 41.81818181818182}
{'Label': "Stephanie's Income", 'MaxFuzzRatio': 100.0}
